# Data Engneering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc
warnings.filterwarnings('ignore')

train = pd.read_csv('./train.csv')
test = pd.read_csv('./testA.csv')

## 1. Pre-processing

In [2]:
numerical_fea = list(train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea, 
                           list(train.select_dtypes(include=['object']).columns)))
label = 'isDefault'
numerical_fea.remove(label)


#check NaN values
train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           1
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n3                  

### 1.1 Dealing with missing values

In [3]:
#Fill numeric features with median
train[numerical_fea] = train[numerical_fea].fillna(train[numerical_fea].median())
test[numerical_fea] = test[numerical_fea].fillna(train[numerical_fea].median())

#Fill categorical features with mode
train[category_fea] = train[category_fea].fillna(train[category_fea].mode())
test[category_fea] = test[category_fea].fillna(train[category_fea].mode())

train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           0
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  0
regionCode                0
dti                       0
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies        0
revolBal                  0
revolUtil                 0
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     0
policyCode                0
n0                        0
n1                        0
n2                        0
n3                  

### 1.2 Catigorical data pre-processing

In [4]:
category_fea

['grade', 'subGrade', 'employmentLength', 'issueDate', 'earliesCreditLine']

#### 1.2.1 grade

In [5]:
# map grade object into int
for data in [train, test]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})

#### 1.2.2 employmentLength

In [6]:
train['employmentLength'].value_counts(dropna=False).sort_index()

employmentLength
1 year        52489
10+ years    262753
2 years       72358
3 years       64152
4 years       47985
5 years       50102
6 years       37254
7 years       35407
8 years       36192
9 years       30272
< 1 year      64237
NaN           46799
Name: count, dtype: int64

In [7]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
    
for data in [train, test]:
    data['employmentLength'].replace('10+ years','10',inplace=True)
    data['employmentLength'].replace('< 1 year','0',inplace=True)
    data['employmentLength']=data['employmentLength'].apply(employmentLength_to_int)
    
train['employmentLength'].value_counts(dropna=False).sort_index()

employmentLength
0.0      64237
1.0      52489
2.0      72358
3.0      64152
4.0      47985
5.0      50102
6.0      37254
7.0      35407
8.0      36192
9.0      30272
10.0    262753
NaN      46799
Name: count, dtype: int64

#### 1.2.3 issueDate

In [8]:
train['issueDate'].head()

0    2014-07-01
1    2012-08-01
2    2015-10-01
3    2015-08-01
4    2016-03-01
Name: issueDate, dtype: object

In [9]:
for data in [train, test]:
    data['issueDate'] = pd.to_datetime(data['issueDate'], format = '%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

#### 1.2.4 earliesCreditLine

In [10]:
train['earliesCreditLine']

0         Aug-2001
1         May-2002
2         May-2006
3         May-1999
4         Aug-1977
            ...   
799995    Aug-2011
799996    May-1989
799997    Jul-2002
799998    Jan-1994
799999    Feb-2002
Name: earliesCreditLine, Length: 800000, dtype: object

In [11]:
import calendar

def time2num(d):
    # d = d['earliesCreditLine']
    string = d[:]
    s1 = list(calendar.month_abbr).index(string[:3])
    if s1 < 10: s1 = '0' + str(s1)
    else: s1 = str(s1)
    # print(string[-4:], s1)
    return np.int64(string[-4:] + s1)


In [12]:
# time2num(train['earliesCreditLine'][0])
for data in [train, test]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(time2num)

### 1.3 Process outliers

In [13]:
# function to find outlires
def find_outliers_by_3sigma(data,fea):
    data_std = np.std(data[fea])
    data_mean = np.mean(data[fea])
    outliers_cut_off = data_std *3
    lower_rule = data_mean - outliers_cut_off
    upper_rule = data_mean + outliers_cut_off
    data[fea+'_outliers'] = data[fea].apply(lambda x:str('abnormal') 
                                           if x < lower_rule or x > upper_rule else 'normal')
    return data

# check the dist of normal and abnormal features
for fea in numerical_fea:
    train = find_outliers_by_3sigma(train,fea)
    print(train[fea+'_outliers'].value_counts())
    print('-'*10)
    print(train.groupby(fea+'_outliers')['isDefault'].sum())
    print('*'*15)

id_outliers
normal    800000
Name: count, dtype: int64
----------
id_outliers
normal    159610
Name: isDefault, dtype: int64
***************
loanAmnt_outliers
normal    800000
Name: count, dtype: int64
----------
loanAmnt_outliers
normal    159610
Name: isDefault, dtype: int64
***************
term_outliers
normal    800000
Name: count, dtype: int64
----------
term_outliers
normal    159610
Name: isDefault, dtype: int64
***************
interestRate_outliers
normal      794259
abnormal      5741
Name: count, dtype: int64
----------
interestRate_outliers
abnormal      2916
normal      156694
Name: isDefault, dtype: int64
***************
installment_outliers
normal      792046
abnormal      7954
Name: count, dtype: int64
----------
installment_outliers
abnormal      2152
normal      157458
Name: isDefault, dtype: int64
***************
employmentTitle_outliers
normal    800000
Name: count, dtype: int64
----------
employmentTitle_outliers
normal    159610
Name: isDefault, dtype: int64
******

In [14]:

for fea in numerical_fea:
    test = find_outliers_by_3sigma(test,fea)
    print(test[fea+'_outliers'].value_counts())
    print('*'*15)

id_outliers
normal    200000
Name: count, dtype: int64
***************
loanAmnt_outliers
normal    200000
Name: count, dtype: int64
***************
term_outliers
normal    200000
Name: count, dtype: int64
***************
interestRate_outliers
normal      198545
abnormal      1455
Name: count, dtype: int64
***************
installment_outliers
normal      198072
abnormal      1928
Name: count, dtype: int64
***************
employmentTitle_outliers
normal    200000
Name: count, dtype: int64
***************
homeOwnership_outliers
normal      199938
abnormal        62
Name: count, dtype: int64
***************
annualIncome_outliers
normal      198900
abnormal      1100
Name: count, dtype: int64
***************
verificationStatus_outliers
normal    200000
Name: count, dtype: int64
***************
purpose_outliers
normal      195747
abnormal      4253
Name: count, dtype: int64
***************
postCode_outliers
normal      199748
abnormal       252
Name: count, dtype: int64
***************
regio

In [15]:
#del outliers
for fea in numerical_fea:
    train = train[train[fea+'_outliers'] == 'normal']
    train = train.reset_index(drop=True)
    train = train.drop([fea+'_outliers'], axis=1)

In [16]:
#del outliers
for fea in numerical_fea:
    test = test[test[fea+'_outliers'] == 'normal']
    test = test.reset_index(drop=True)
    test = test.drop([fea+'_outliers'], axis=1)

### 1.3 Data mapping

In [17]:
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership',
                'verificationStatus', 'purpose', 'postCode','regionCode',
                'applicationType', 'initialListStatus', 'title', 'policyCode']

for i in cate_features:
    print(i,'types:', train[i].nunique())

grade types: 7
subGrade types: 35
employmentTitle types: 197936
homeOwnership types: 3
verificationStatus types: 3
purpose types: 9
postCode types: 858
regionCode types: 50
applicationType types: 1
initialListStatus types: 2
title types: 15440
policyCode types: 1


### 1.3.1 low-dim features

In [18]:
#types>2 and not so big, use one-hot
train = pd.get_dummies(train, columns=['subGrade', 'homeOwnership', 'purpose', 'regionCode','verificationStatus'], drop_first=True)

In [19]:
test = pd.get_dummies(test,columns=['subGrade', 'homeOwnership', 'purpose', 'regionCode','verificationStatus'],drop_first=True)

### 1.3.2 high-dim features

In [20]:
for f in ['employmentTitle', 'title', 'postCode']:
    train[f+'_cnts'] = train.groupby([f])['id'].transform('count')
    train[f+'_rank'] = train.groupby([f])['id'].rank(ascending=False).astype(int)
    del train[f]

In [21]:
for f in ['employmentTitle','title', 'postCode']:
    test[f+'_cnts'] = test.groupby([f])['id'].transform('count')
    test[f+'_rank'] = test.groupby([f])['id'].rank(ascending=False).astype(int)
    del test[f]

In [22]:
train.to_csv('clean_train.csv')
test.to_csv('clean_test.csv')